# 数据合并处理

In [11]:
import pandas as pd
import numpy as np
import scipy as sp

def read_csv_file(f, logging = False):
    data = pd.read_csv(f)
    if logging:
        print(data.head())
        print(data.columns.values)
        print(data.describe())
        print(data.info())
    return data

def categories_process_first_class(cate):
    cate = str(cate)
    return int(cate[0])
        
    
def categories_process_second_class(cate):
    cate = str(cate)
    if len(cate) < 3:
        return 0
    else:
        return int(cate[1:])
    
def age_process(age):
    age = int(age)
    if age == 0:
        return 0
    if age < 16:
        return 1
    if age < 23:
        return 2
    if age < 27:
        return 3
    else:
        return 4

def province_process(hometown):
    hometown = str(hometown)
    if len(hometown) == 1:
        return 0
    else:
        return int(hometown[: -2])

def city_process(hometown):
    hometown = str(hometown)
    if len(hometown) == 1:
        return 0
    else:
        return int(hometown[-2: ])

def get_time_day(t):
    t = str(t)
    t = int(t[0: 2])
    return t

def get_time_hour(t):
    t = str(t)
    t = int(t[2: 4])
    if t < 6:
        return 0
    if t < 12:
        return 1
    if t < 18:
        return 2
    else:
        return 3

def get_time_min(t):
    t = str(t)
    t = int(t[4: ])
    return t

def logloss(act, pred):
    epsilon = 1e-5
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1 - eplison, pred)
    ll = sum(act * sp.log(pred) + sp.subtract(1, act) * sp.log(sp.subtract(1, pred)))
    ll = ll * -1.0/len(act)
    return ll

In [12]:
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

print('test dataset')
test_data = read_csv_file('./pre/test.csv', logging = True)
print('================================================\n')
print('train dataset:')
train_data = read_csv_file('./pre/train.csv', logging = True)
print('================================================\n')
print('ad dataset')
ad = read_csv_file('./pre/ad.csv', logging = True)
print('================================================\n')
print('app dataset')
app_categories = read_csv_file('./pre/app_categories.csv', logging = True)
print('================================================\n')
print('user dataset')
user = read_csv_file('./pre/user.csv', logging = True)
print('================================================\n')
print('position dataset')
position = read_csv_file('./pre/position.csv', logging= True)
print('================================================\n')
print('user_app_actions dataset')
user_app_actions = read_csv_file('./pre/user_app_actions.csv', logging = True)

test dataset
   instanceID  label  clickTime  creativeID   userID  positionID  \
0           1     -1     310000        3745  1164848        3451   
1           2     -1     310000        2284  2127247        1613   
2           3     -1     310000        1456  2769125        5510   
3           4     -1     310000        4565     9762        4113   
4           5     -1     310000          49  2513636        3615   

   connectionType  telecomsOperator  
0               1                 3  
1               1                 3  
2               2                 1  
3               2                 3  
4               1                 3  
['instanceID' 'label' 'clickTime' 'creativeID' 'userID' 'positionID'
 'connectionType' 'telecomsOperator']
          instanceID     label      clickTime     creativeID        userID  \
count  338489.000000  338489.0  338489.000000  338489.000000  3.384890e+05   
mean   169245.000000      -1.0  311479.490613    3001.534765  1.409519e+06   
std     9

   userID  installTime  appID
0       1       182032    933
1       5       141354    589
2       5       251727    565
3       5       260932    596
4       5       261318    536
['userID' 'installTime' 'appID']
             userID   installTime         appID
count  6.003471e+06  6.003471e+06  6.003471e+06
mean   1.403576e+06  1.483695e+05  5.125716e+03
std    8.074011e+05  9.097316e+04  2.118739e+04
min    1.000000e+00  1.000000e+04  3.540000e+02
25%    7.055445e+05  6.164500e+04  5.890000e+02
50%    1.404560e+06  1.408000e+05  8.270000e+02
75%    2.100952e+06  2.320160e+05  1.969000e+03
max    2.805117e+06  3.023590e+05  4.332670e+05
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6003471 entries, 0 to 6003470
Data columns (total 3 columns):
userID         int64
installTime    int64
appID          int64
dtypes: int64(3)
memory usage: 137.4 MB
None


In [ ]:
print('================================================\n')
print('user_installedapps dataset')
user_installedapps = read_csv_file('./pre/user_installedapps.csv', logging = False)

In [ ]:
train = train_data.drop(['label', 'conversionTime'], axis = 1)
test = test_data.drop(['label', 'instanceID'], axis = 1)
data = pd.concat((train, test), axis = 0)

In [ ]:
print(data.shape)
print(train.shape)
print(test.shape)

In [ ]:
data.isnull().sum()

In [ ]:
data_ad = data.merge(ad, on = 'creativeID', how = 'left')
data_ad.shape

In [ ]:
data_ad['click_day'] = data_ad['clickTime'].apply(get_time_day)
data_ad['click_min'] = data_ad['clickTime'].apply(get_time_min)
data_ad['click_hour'] = data_ad['clickTime'].apply(get_time_hour)
data_ad.drop(['clickTime'], inplace = True, axis = 1)
data_ad.tail()

In [ ]:
data_ad_app = data_ad.merge(app_categories, on = 'appID', how = 'left')
data_ad_app['app_first_categories'] = data_ad_app['appCategory'].apply(categories_process_first_class)
data_ad_app['app_second_categories'] = data_ad_app['appCategory'].apply(categories_process_second_class)
data_ad_app.drop(['appCategory'], axis = 1, inplace = True)

In [ ]:
data_ad_app_user = data_ad_app.merge(user, on = 'userID', how = 'left')

In [ ]:
##test数据集的age数据缺失较多
data_ad_app_user['age'].describe()

In [ ]:
data_ad_app_user['age'].replace(0.0, data_ad_app_user['age'][data_ad_app_user['age'] != 0.0].mean(), inplace = True)

In [ ]:
data_ad_app_user['age'].fillna(data_ad_app_user['age'][data_ad_app_user['age'] != 0.0].mean(), inplace = True)
data_ad_app_user['age'].isnull().sum()

In [ ]:
data_ad_app_user['age'] = data_ad_app_user['age'].apply(age_process)

In [ ]:
data_ad_app_user['residence_province'] = data_ad_app_user['residence'].apply(province_process)
data_ad_app_user['residence_city'] = data_ad_app_user['residence'].apply(city_process)
data_ad_app_user.drop(['residence'], inplace = True, axis = 1)
data_ad_app_user.tail()

In [ ]:
data_ad_app_user_position = data_ad_app_user.merge(position, on = 'positionID', how = 'left')

In [ ]:
x_train = data_ad_app_user_position[: 3749528]
y_train = train_data['label']
test = data_ad_app_user_position[3749528: ]

In [ ]:
import pickle
with open('values.pkl', 'wb') as f:
    pickle.dump(x_train, f)
    pickle.dump(y_train, f)
    pickle.dump(test, f)

# 数据建模

## 导入数据

In [7]:
import pickle
with open('values.pkl', 'rb') as f:
    x_train = pickle.load(f)
    y_train = pickle.load(f)
    test = pickle.load(f) 

In [ ]:
y_train.value_counts()
##正负样本非常不均衡，但由于过采样后数据量过大，电脑内存有限，使用欠采样的方法

## 过采样后存储数据

In [ ]:
from imblearn.over_sampling import ADASYN

x_train_oversam, y_train_oversam = ADASYN().fit_sample(x_train, y_train)

In [ ]:
import pickle
with open('values_resam.pkl', 'wb') as f:
    pickle.dump(x_train_oversam, f)
    pickle.dump(y_train_oversam, f)   

In [ ]:
print(x_train_oversam.shape)
print(y_train_oversam.shape)

In [ ]:
## EasyEnsemble
'''
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold

bbc = BalancedBaggingClassifier(
    base_estimator = DecisionTreeClassifier(),
    ratio = 'auto',
    replacement = False,
    random_state = 100,
    n_jobs = -1
)

kf = KFold(n_splits = 5)
for train_index, val_index in kf.split(x_train, y_train):
    X_train,Y_train = x_train[train_index, : ], y_train[train_index]
    X_val, Y_val = x_train[val_index, : ], y_train[val_index]
    bbc.fit(X_train, Y_train)
    pred_bbc = bbc.predict_proba(X_val)
    print(pred_bbc)
    print('The logloss is ', logloss(Y_val, pred_bbc))
'''

## 欠采样并存储数据

In [ ]:
import numpy as np
import pickle
from imblearn.under_sampling import RandomUnderSampler

with open('values.pkl', 'rb') as f:
    x_train = pickle.load(f)
    y_train = pickle.load(f)
    test = pickle.load(f)
    
#x_train = np.array(x_train)
#y_train = np.array(y_train)
#test = np.array(test)

In [18]:
y_train = train_data['label']
y_train = pd.DataFrame(y_train, columns = ['label'])

In [21]:
from imblearn.under_sampling import RandomUnderSampler

bc = RandomUnderSampler(random_state = 100)
x_train_resam, y_train_resam = bc.fit_sample(x_train, y_train)

with open('values_undersampling.pkl', 'wb') as f:
    pickle.dump(x_train_resam, f)
    pickle.dump(y_train_resam, f)
    pickle.dump(test, f)

## 载入数据

In [22]:
import pickle

with open('values_undersampling.pkl', 'rb') as f:
    x_train_resam = pickle.load(f)
    y_train_resam = pickle.load(f)
    test = pickle.load(f)

In [23]:
print(x_train_resam)

[[   4565  156503    3789 ...       8       0       1]
 [   4565 2159098    2891 ...      21       0       1]
 [   2808 1140159    2868 ...       4       0       1]
 ...
 [   1675 1445513    2188 ...       1       0       1]
 [   4077 1252796    5539 ...       0       1       0]
 [   2038 2216718    1865 ...       6       0       2]]


### Xgbosst

#### 用sklearn API

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {
    'max_depth': [3, 4, 5, 8, 10],
    'n_estimators': [50, 100, 200, 400, 600, 800, 1000],
    'laerning_rate': [0.1, 0.2, 0.3],
    'gamma': [0, 0.2],
    'subsample': [0.8, 1],
             }

xgb_model = xgb.XGBClassifier()
rgs = GridSearchCV(xgb_model, param_grid, n_jobs = -1, cv = 5)
rgs.fit(x_train_resam, y_train_resam)
print(rgs.best_score_)
print(rgs.best_params_)
pred = rgs.predict_proba(test)
print('The logloss is ', logloss(Y_val, pred_bbc))

#### 用Python API

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(x_train_resam, label = y_train_resam)
num_xgb = 500

params = {
    'booster': 'gbtree',
    'eta': 0.2, 
    'eval_metric': 'logloss',
    'tree method': 'auto',
    'objective': 'binary:logistic'
}

bst = xgb.cv(params, dtrain, num_xgb, nfold = 5, verbose_eval = True)

In [ ]:
dtest = xgb.DMatrix(test)
pred = bst.predict(dtest)
print(logloss()

In [24]:
x_train_resam = pd.DataFrame(x_train_resam, columns = x_train.columns)
y_train_resam = pd.DataFrame(y_train_resam, columns = ['label'])
print(x_train_resam)

        creativeID   userID  positionID  connectionType  telecomsOperator  \
0             4565   156503        3789               2                 3   
1             4565  2159098        2891               1                 1   
2             2808  1140159        2868               1                 1   
3             6241  2495735        1037               1                 3   
4             2635  2681266        5539               1                 2   
5             5187   698746        3150               3                 3   
6             4503   554932        3322               1                 2   
7              863  1654378        7503               3                 1   
8             4511   881257        3322               4                 3   
9             5950   716116        3412               1                 1   
10            5950  2007531        3293               1                 1   
11            4077   258407        3284               1                 0   

In [25]:
import pickle

with open('lgb_values.pkl', 'wb') as f:
    pickle.dump(x_train_resam, f)
    pickle.dump(y_train_resam, f)
    pickle.dump(test, f)

### LightGBM

In [27]:
import pickle

with open('lgb_values.pkl', 'rb') as f:
    x_train_resam = pickle.load(f)
    y_trian_resam = pickle.load(f)
    test = pickle.load(f)

In [45]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [49]:
X_train, X_val, y_train, y_val = train_test_split(x_train_resam, y_train_resam, test_size = 0.2)
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary', 
    'metric': 'binary_logloss',
    'num_leaves': 3,
    'max_depth': 10,
    'min_data_in_leaf': 40,
    'learning_rate': 0.5,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l2': 0.001,
    'verbose': 1
}


lgb_train = lgb.Dataset(X_train, label = y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference = lgb_train)
gbm = lgb.train(params, lgb_train, num_boost_round = 2000, verbose_eval = True, valid_sets = lgb_val, early_stopping_rounds = None)
pred = gbm.predict(test, num_iteration = gbm.best_iteration)

importance = gbm.feature_importance()
names = gbm.feature_name()
for idx, im in enumerate(importance):
    print('The score of {} is {}'.format(names[idx], im))

[1]	valid_0's binary_logloss: 0.663285
[2]	valid_0's binary_logloss: 0.649055
[3]	valid_0's binary_logloss: 0.64207
[4]	valid_0's binary_logloss: 0.636542
[5]	valid_0's binary_logloss: 0.631981
[6]	valid_0's binary_logloss: 0.628963
[7]	valid_0's binary_logloss: 0.627012
[8]	valid_0's binary_logloss: 0.62525
[9]	valid_0's binary_logloss: 0.623768
[10]	valid_0's binary_logloss: 0.622311
[11]	valid_0's binary_logloss: 0.619436
[12]	valid_0's binary_logloss: 0.617155
[13]	valid_0's binary_logloss: 0.615936
[14]	valid_0's binary_logloss: 0.614226
[15]	valid_0's binary_logloss: 0.612698
[16]	valid_0's binary_logloss: 0.611778
[17]	valid_0's binary_logloss: 0.610806
[18]	valid_0's binary_logloss: 0.610087
[19]	valid_0's binary_logloss: 0.608682
[20]	valid_0's binary_logloss: 0.607939
[21]	valid_0's binary_logloss: 0.607441
[22]	valid_0's binary_logloss: 0.605744
[23]	valid_0's binary_logloss: 0.605033
[24]	valid_0's binary_logloss: 0.60443
[25]	valid_0's binary_logloss: 0.603894
[26]	valid_0

[212]	valid_0's binary_logloss: 0.570979
[213]	valid_0's binary_logloss: 0.571022
[214]	valid_0's binary_logloss: 0.570909
[215]	valid_0's binary_logloss: 0.570924
[216]	valid_0's binary_logloss: 0.570824
[217]	valid_0's binary_logloss: 0.570852
[218]	valid_0's binary_logloss: 0.570889
[219]	valid_0's binary_logloss: 0.570866
[220]	valid_0's binary_logloss: 0.570864
[221]	valid_0's binary_logloss: 0.570755
[222]	valid_0's binary_logloss: 0.570745
[223]	valid_0's binary_logloss: 0.57069
[224]	valid_0's binary_logloss: 0.570721
[225]	valid_0's binary_logloss: 0.570723
[226]	valid_0's binary_logloss: 0.570753
[227]	valid_0's binary_logloss: 0.570546
[228]	valid_0's binary_logloss: 0.570546
[229]	valid_0's binary_logloss: 0.570531
[230]	valid_0's binary_logloss: 0.570562
[231]	valid_0's binary_logloss: 0.570339
[232]	valid_0's binary_logloss: 0.57041
[233]	valid_0's binary_logloss: 0.570401
[234]	valid_0's binary_logloss: 0.570392
[235]	valid_0's binary_logloss: 0.570404
[236]	valid_0's bi

[428]	valid_0's binary_logloss: 0.56519
[429]	valid_0's binary_logloss: 0.565204
[430]	valid_0's binary_logloss: 0.565237
[431]	valid_0's binary_logloss: 0.565157
[432]	valid_0's binary_logloss: 0.565186
[433]	valid_0's binary_logloss: 0.565197
[434]	valid_0's binary_logloss: 0.565164
[435]	valid_0's binary_logloss: 0.565166
[436]	valid_0's binary_logloss: 0.565123
[437]	valid_0's binary_logloss: 0.565105
[438]	valid_0's binary_logloss: 0.565103
[439]	valid_0's binary_logloss: 0.565073
[440]	valid_0's binary_logloss: 0.565048
[441]	valid_0's binary_logloss: 0.565028
[442]	valid_0's binary_logloss: 0.565043
[443]	valid_0's binary_logloss: 0.565031
[444]	valid_0's binary_logloss: 0.565033
[445]	valid_0's binary_logloss: 0.564977
[446]	valid_0's binary_logloss: 0.564955
[447]	valid_0's binary_logloss: 0.56497
[448]	valid_0's binary_logloss: 0.564992
[449]	valid_0's binary_logloss: 0.564987
[450]	valid_0's binary_logloss: 0.565017
[451]	valid_0's binary_logloss: 0.564939
[452]	valid_0's bi

[643]	valid_0's binary_logloss: 0.563082
[644]	valid_0's binary_logloss: 0.563064
[645]	valid_0's binary_logloss: 0.563064
[646]	valid_0's binary_logloss: 0.563038
[647]	valid_0's binary_logloss: 0.563063
[648]	valid_0's binary_logloss: 0.563116
[649]	valid_0's binary_logloss: 0.56309
[650]	valid_0's binary_logloss: 0.563069
[651]	valid_0's binary_logloss: 0.563103
[652]	valid_0's binary_logloss: 0.56314
[653]	valid_0's binary_logloss: 0.56313
[654]	valid_0's binary_logloss: 0.563136
[655]	valid_0's binary_logloss: 0.563111
[656]	valid_0's binary_logloss: 0.56312
[657]	valid_0's binary_logloss: 0.56312
[658]	valid_0's binary_logloss: 0.563112
[659]	valid_0's binary_logloss: 0.563154
[660]	valid_0's binary_logloss: 0.563111
[661]	valid_0's binary_logloss: 0.563084
[662]	valid_0's binary_logloss: 0.563073
[663]	valid_0's binary_logloss: 0.563081
[664]	valid_0's binary_logloss: 0.563088
[665]	valid_0's binary_logloss: 0.563069
[666]	valid_0's binary_logloss: 0.563088
[667]	valid_0's binar

[845]	valid_0's binary_logloss: 0.562206
[846]	valid_0's binary_logloss: 0.562164
[847]	valid_0's binary_logloss: 0.562137
[848]	valid_0's binary_logloss: 0.562196
[849]	valid_0's binary_logloss: 0.562214
[850]	valid_0's binary_logloss: 0.562215
[851]	valid_0's binary_logloss: 0.562228
[852]	valid_0's binary_logloss: 0.562233
[853]	valid_0's binary_logloss: 0.562226
[854]	valid_0's binary_logloss: 0.562271
[855]	valid_0's binary_logloss: 0.562257
[856]	valid_0's binary_logloss: 0.562261
[857]	valid_0's binary_logloss: 0.562257
[858]	valid_0's binary_logloss: 0.56228
[859]	valid_0's binary_logloss: 0.562274
[860]	valid_0's binary_logloss: 0.562284
[861]	valid_0's binary_logloss: 0.562238
[862]	valid_0's binary_logloss: 0.56226
[863]	valid_0's binary_logloss: 0.562284
[864]	valid_0's binary_logloss: 0.562283
[865]	valid_0's binary_logloss: 0.562289
[866]	valid_0's binary_logloss: 0.562286
[867]	valid_0's binary_logloss: 0.56228
[868]	valid_0's binary_logloss: 0.562284
[869]	valid_0's bin

[1051]	valid_0's binary_logloss: 0.562317
[1052]	valid_0's binary_logloss: 0.562342
[1053]	valid_0's binary_logloss: 0.562291
[1054]	valid_0's binary_logloss: 0.562273
[1055]	valid_0's binary_logloss: 0.56228
[1056]	valid_0's binary_logloss: 0.562268
[1057]	valid_0's binary_logloss: 0.562313
[1058]	valid_0's binary_logloss: 0.562283
[1059]	valid_0's binary_logloss: 0.562275
[1060]	valid_0's binary_logloss: 0.562272
[1061]	valid_0's binary_logloss: 0.562265
[1062]	valid_0's binary_logloss: 0.562286
[1063]	valid_0's binary_logloss: 0.562274
[1064]	valid_0's binary_logloss: 0.562239
[1065]	valid_0's binary_logloss: 0.562175
[1066]	valid_0's binary_logloss: 0.56216
[1067]	valid_0's binary_logloss: 0.562152
[1068]	valid_0's binary_logloss: 0.562151
[1069]	valid_0's binary_logloss: 0.562164
[1070]	valid_0's binary_logloss: 0.562173
[1071]	valid_0's binary_logloss: 0.562156
[1072]	valid_0's binary_logloss: 0.562177
[1073]	valid_0's binary_logloss: 0.562154
[1074]	valid_0's binary_logloss: 0.5

[1259]	valid_0's binary_logloss: 0.562199
[1260]	valid_0's binary_logloss: 0.562245
[1261]	valid_0's binary_logloss: 0.562244
[1262]	valid_0's binary_logloss: 0.562248
[1263]	valid_0's binary_logloss: 0.562245
[1264]	valid_0's binary_logloss: 0.56224
[1265]	valid_0's binary_logloss: 0.56226
[1266]	valid_0's binary_logloss: 0.562291
[1267]	valid_0's binary_logloss: 0.562303
[1268]	valid_0's binary_logloss: 0.562302
[1269]	valid_0's binary_logloss: 0.562299
[1270]	valid_0's binary_logloss: 0.562272
[1271]	valid_0's binary_logloss: 0.562264
[1272]	valid_0's binary_logloss: 0.562236
[1273]	valid_0's binary_logloss: 0.562219
[1274]	valid_0's binary_logloss: 0.562263
[1275]	valid_0's binary_logloss: 0.562255
[1276]	valid_0's binary_logloss: 0.562245
[1277]	valid_0's binary_logloss: 0.562205
[1278]	valid_0's binary_logloss: 0.562192
[1279]	valid_0's binary_logloss: 0.56222
[1280]	valid_0's binary_logloss: 0.562257
[1281]	valid_0's binary_logloss: 0.562141
[1282]	valid_0's binary_logloss: 0.56

[1475]	valid_0's binary_logloss: 0.562038
[1476]	valid_0's binary_logloss: 0.562049
[1477]	valid_0's binary_logloss: 0.56207
[1478]	valid_0's binary_logloss: 0.562072
[1479]	valid_0's binary_logloss: 0.562092
[1480]	valid_0's binary_logloss: 0.562127
[1481]	valid_0's binary_logloss: 0.562122
[1482]	valid_0's binary_logloss: 0.562085
[1483]	valid_0's binary_logloss: 0.562125
[1484]	valid_0's binary_logloss: 0.562166
[1485]	valid_0's binary_logloss: 0.56216
[1486]	valid_0's binary_logloss: 0.56218
[1487]	valid_0's binary_logloss: 0.562151
[1488]	valid_0's binary_logloss: 0.562165
[1489]	valid_0's binary_logloss: 0.562155
[1490]	valid_0's binary_logloss: 0.562168
[1491]	valid_0's binary_logloss: 0.56219
[1492]	valid_0's binary_logloss: 0.562182
[1493]	valid_0's binary_logloss: 0.562201
[1494]	valid_0's binary_logloss: 0.562196
[1495]	valid_0's binary_logloss: 0.562195
[1496]	valid_0's binary_logloss: 0.562189
[1497]	valid_0's binary_logloss: 0.56222
[1498]	valid_0's binary_logloss: 0.5622

[1672]	valid_0's binary_logloss: 0.562204
[1673]	valid_0's binary_logloss: 0.562194
[1674]	valid_0's binary_logloss: 0.562192
[1675]	valid_0's binary_logloss: 0.562202
[1676]	valid_0's binary_logloss: 0.562217
[1677]	valid_0's binary_logloss: 0.562213
[1678]	valid_0's binary_logloss: 0.562201
[1679]	valid_0's binary_logloss: 0.562181
[1680]	valid_0's binary_logloss: 0.562177
[1681]	valid_0's binary_logloss: 0.56219
[1682]	valid_0's binary_logloss: 0.562193
[1683]	valid_0's binary_logloss: 0.562171
[1684]	valid_0's binary_logloss: 0.562182
[1685]	valid_0's binary_logloss: 0.562202
[1686]	valid_0's binary_logloss: 0.562148
[1687]	valid_0's binary_logloss: 0.56212
[1688]	valid_0's binary_logloss: 0.562125
[1689]	valid_0's binary_logloss: 0.562091
[1690]	valid_0's binary_logloss: 0.562097
[1691]	valid_0's binary_logloss: 0.56209
[1692]	valid_0's binary_logloss: 0.562036
[1693]	valid_0's binary_logloss: 0.562036
[1694]	valid_0's binary_logloss: 0.56206
[1695]	valid_0's binary_logloss: 0.562

[1878]	valid_0's binary_logloss: 0.562057
[1879]	valid_0's binary_logloss: 0.56204
[1880]	valid_0's binary_logloss: 0.562038
[1881]	valid_0's binary_logloss: 0.562044
[1882]	valid_0's binary_logloss: 0.562058
[1883]	valid_0's binary_logloss: 0.562053
[1884]	valid_0's binary_logloss: 0.56205
[1885]	valid_0's binary_logloss: 0.562021
[1886]	valid_0's binary_logloss: 0.562042
[1887]	valid_0's binary_logloss: 0.562053
[1888]	valid_0's binary_logloss: 0.562043
[1889]	valid_0's binary_logloss: 0.562034
[1890]	valid_0's binary_logloss: 0.562054
[1891]	valid_0's binary_logloss: 0.56207
[1892]	valid_0's binary_logloss: 0.562112
[1893]	valid_0's binary_logloss: 0.562037
[1894]	valid_0's binary_logloss: 0.562051
[1895]	valid_0's binary_logloss: 0.562074
[1896]	valid_0's binary_logloss: 0.562078
[1897]	valid_0's binary_logloss: 0.562062
[1898]	valid_0's binary_logloss: 0.562069
[1899]	valid_0's binary_logloss: 0.562052
[1900]	valid_0's binary_logloss: 0.562047
[1901]	valid_0's binary_logloss: 0.56